In [79]:
import mat73
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from IPython.display import display
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd 
from os import walk
import json 

In [80]:
def read_json(file_name):
    with open(file_name) as f:
      data = json.load(f)
    return data

In [44]:
b =  [1,   2,   3,  4,   5,    6,  7,  8,    9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
bstress = [22, 10, 19, 17, 23, 14, 9, 18, 23, 10, 8, 13, 12, 20, 21, 13, 0, 28, 13, 7, 18, 13, 15]
g =  [1,    2,   3,  4,  5,  6,  7,  8,    9, 10, 11, 12, 13, 14, 15, 16, 17,]
gstress =  [11, 13, 25, 7, 28, 14, 8, 13, 10, 12, 11, 16, 20, 25, 16, 31, 9]

In [55]:
df = pd.read_csv('../table.csv')

In [56]:
df.columns

Index(['Exp_ID', 'exp_time', 'trais no.', 'note', 'signal_condition', 'gender',
       'age', 'race', 'handiness', 'sleep condition', 'Q11', 'Q12', 'Q13',
       'Q13_r', 'Q14', 'ID', 'ID_boy', 'ID_girl', 'Raw_within_RF',
       'Raw_within_logistic', 'within_classification',
       'within_classification_logistic', 'Raw_within_netVSneg_RF',
       'Raw_within_netVSneg_logistic', 'Within_netVSneg_RF',
       'Within_netVSneg_logistic', 'within_Feature_asymmtry_RF',
       'within_Feature_Less_RF', 'within_Feature_Less_Logistic', 'stress',
       'stressGroup', '42set_ID', 'between_classification_all',
       'Between_Feature_asymmtry', 'Between_Feature_Less',
       'raw_betweenNeutral2Negative', 'raw_StressGroupbetweenNeutral2Negative',
       'raw_between_stressGroup', 'between_stressGroup',
       'Between_Stress2groups_RF', 'Between_Stress2groups_log',
       'Between_GenderGroup', 'Between_RandomGroup',
       'between_stressGroup_netural_vs_negative', '39set_ID',
       'between_

In [181]:
folder = '../results/sleep_condition/'

In [182]:
_, _, filenames = next(walk(folder))

In [183]:
filenames.sort()

In [184]:
filenames

['bad.json', 'good.json']

In [187]:
bad = read_json(folder+filenames[1])['avg_acc']
sum(bad)/len(bad)

0.7727272727272727

In [169]:
num = 0
final = [0 for _ in range(40)]
for num in range(100):
    arr = []
    for group in range(3):
        accs = read_json(folder+f"trailNum{num}_groupNum{group}.json")['avg_acc']
        ids = read_json(folder+f"trailNum{num}_groupNum{group}.json")['ids']
        for i in range(len(accs)):
            arr.append((ids[i],accs[i]))
    arr.sort()
    for j in arr:
        final[j[0]]+=j[1]
print(final)

[75.3222222222222, 73.48333333333336, 70.50000000000003, 72.24166666666667, 73.61666666666665, 72.74444444444445, 71.87500000000001, 71.83055555555553, 71.77500000000002, 72.41111111111114, 71.94722222222225, 71.98333333333336, 70.57500000000003, 70.90833333333335, 72.50277777777777, 71.73055555555557, 75.2333333333334, 72.7138888888889, 72.69999999999999, 68.83888888888887, 71.43888888888888, 73.79722222222222, 69.28055555555557, 72.8388888888889, 72.42222222222223, 73.20000000000002, 72.16111111111111, 71.96666666666664, 71.50833333333331, 71.16388888888889, 72.0527777777778, 71.79166666666666, 70.65277777777779, 70.5277777777778, 71.78611111111111, 72.9527777777778, 72.05555555555557, 71.74722222222223, 72.35000000000002, 71.62222222222225]


In [170]:
for i in range(len(final)):
    final[i] = final[i]/100
print(final)

[0.7532222222222219, 0.7348333333333337, 0.7050000000000003, 0.7224166666666667, 0.7361666666666664, 0.7274444444444446, 0.7187500000000001, 0.7183055555555553, 0.7177500000000002, 0.7241111111111114, 0.7194722222222225, 0.7198333333333337, 0.7057500000000003, 0.7090833333333335, 0.7250277777777776, 0.7173055555555556, 0.7523333333333341, 0.727138888888889, 0.7269999999999999, 0.6883888888888887, 0.7143888888888889, 0.7379722222222221, 0.6928055555555557, 0.728388888888889, 0.7242222222222223, 0.7320000000000002, 0.7216111111111111, 0.7196666666666665, 0.7150833333333331, 0.7116388888888889, 0.7205277777777781, 0.7179166666666665, 0.7065277777777779, 0.705277777777778, 0.7178611111111111, 0.729527777777778, 0.7205555555555557, 0.7174722222222223, 0.7235000000000003, 0.7162222222222225]


In [188]:
df = pd.read_csv('../table.csv')

In [200]:
vals = df['Q13_r'].values 
net =[]
neg = []
print(type(vals[0]))
for i in range(len(vals)):
    temp = str(vals[i])
    if 'Neg' in temp:
        neg.append(i)
    elif 'Neu' in temp:
        net.append(i)
print(net,neg)

<class 'str'>
[2, 3, 5, 8, 9, 10, 11, 12, 16, 18, 22, 23, 24, 27, 28, 29, 30, 32, 33, 36, 37, 38, 39, 41] [0, 4, 6, 7, 14, 15, 19, 20, 21, 25, 26, 31, 34]


In [172]:
for _ in range(20):
    final.append(-1)
df['Between_RandomGroup'] = final[:52]
df.to_csv('../table.csv')

In [138]:
overall.sort(key=lambda x:x[0])
good.sort(key=lambda x:x[0])

In [151]:
print(overall)

[0.8888888888888888, 0.8333333333333334, 0.8611111111111112, 0.75, 0.8333333333333334, 0.8888888888888888, 0.8611111111111112, 0.6666666666666666, 0.6388888888888888, 0.75, 0.7666666666666667, 0.5666666666666667, 0.8888888888888888, 0.7333333333333333, 0.5, 0.7222222222222222, 0.8055555555555556, 0.6388888888888888, 0.5833333333333334, 0.6944444444444444, 0.75, 0.5833333333333334, 0.6666666666666666, 0.6944444444444444, 0.8333333333333334, 0.7222222222222222, 0.8611111111111112, 0.5, 0.6944444444444444, 0.6111111111111112, 0.8888888888888888, 0.8055555555555556, 0.7777777777777778, 0.6944444444444444, 0.5833333333333334, 0.8611111111111112, 0.7777777777777778, 0.75, 0.8333333333333334, 0.8, 0.6333333333333333, 0.5666666666666667]


In [140]:
for i in range(len(overall)):
    overall[i] = overall[i][1]

In [179]:
overall = read_json(folder+filenames[1])['avg_acc']
print(overall)

[0.7777777777777778, 0.7222222222222222, 0.2777777777777778, 0.2777777777777778, 0.5, 0.2777777777777778, 0.6666666666666666, 0.6666666666666666, 0.5, 0.4444444444444444, 0.6333333333333333, 0.6666666666666666, 0.6111111111111112, 0.5333333333333333, 0.4444444444444444, 0.3611111111111111, 0.5833333333333334, 0.5277777777777778, 0.4583333333333333, 0.25, 0.6666666666666666, 0.3055555555555556, 0.7222222222222222, 0.7777777777777778, 0.5277777777777778, 0.3055555555555556, 0.9166666666666666, 0.6111111111111112, 0.6111111111111112, 0.5, 0.6388888888888888, 0.5833333333333334, 0.5555555555555556, 0.5277777777777778, 0.3333333333333333, 0.5, 0.6666666666666666, 0.6944444444444444, 0.4444444444444444, 0.5666666666666667, 0.3333333333333333, 0.36666666666666664, 0.25, 0.6666666666666666, 0.2777777777777778, 0.75]


In [180]:
for _ in range(20):
    overall.append(-1)
df['Raw_within_netVSneg_RF'] = overall[:52]
df.to_csv('../table.csv')